In [2]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/home/kirill/Desktop/regex_by_comm.csv')
data.head()

,Unnamed: 0,code_block,tag,imports,load_data,visualize,preprocessing,model,train,predict
0,0,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,"[' plotting', ' linear algebra', ' accessing d...",0,1,1,0,0,0,0
1,1,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,"[' plotting', ' linear algebra', ' accessing d...",0,1,1,0,0,0,0
2,2,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,"[' plotting', ' linear algebra', ' accessing d...",0,1,1,0,0,0,0
3,3,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,"[' plotting', ' linear algebra', ' accessing d...",0,1,1,0,0,0,0
4,4,from sklearn.preprocessing import StandardScal...,"[' plotting', ' linear algebra', ' accessing d...",0,1,1,0,0,0,0


Уникальных тегов:

In [3]:
data['tag'].nunique()

64975

Размер:

In [4]:
data['tag'].shape

(107328,)

In [5]:
from collections import Counter
from tqdm import tqdm

In [6]:
data = data.drop(columns='Unnamed: 0', axis=1)

Убираем данные без какой либо метки

In [13]:
unlabeled_data = data[data['imports'] + data['load_data'] + data['preprocessing'] + data['model'] + data['train'] + data['predict'] == 0]

In [14]:
data = data.drop(index=unlabeled_data.index)

In [17]:
ready_data = data
ready_data = ready_data.drop(columns=['tag'], axis=1)
ready_data

,code_block,imports,load_data,visualize,preprocessing,model,train,predict
0,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,0,1,1,0,0,0,0
1,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,0,1,1,0,0,0,0
2,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,0,1,1,0,0,0,0
3,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,0,1,1,0,0,0,0
4,from sklearn.preprocessing import StandardScal...,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
107323,"\nclf_list = [('Perceptron', Perceptron(eta0=0...",0,0,1,0,0,0,1
107324,"def test_set_performances(clf_tuple_list, X_tr...",0,0,0,0,0,1,1
107325,"\nmodel_names, test_acc, test_f1 = test_set_pe...",0,0,0,0,0,0,1
107326,"def plot_confusion_matrix(true_y, pred_y, titl...",0,0,1,0,0,1,0


In [18]:
to_drop = ready_data[ready_data['code_block'].str.len() > 10000].index

ready_data = ready_data.drop(index=to_drop)

In [20]:
ready_data = ready_data.rename(columns={"load_data":"loaddata"})

In [23]:
ready_data = ready_data.reset_index().drop(columns=['index'], axis=1)

In [22]:
to_del = []
for i in range(ready_data.shape[0]):
    if 'def' in ready_data.loc[i, 'code_block']:
        to_del.append(i)

ready_data = ready_data.drop(index=to_del, axis=1)
ready_data = ready_data.reset_index().drop(columns=['index'], axis=1)
ready_data

,code_block,imports,loaddata,visualize,preprocessing,model,train,predict
0,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,0,1,1,0,0,0,0
1,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,0,1,1,0,0,0,0
2,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,0,1,1,0,0,0,0
3,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,0,1,1,0,0,0,0
4,from sklearn.preprocessing import StandardScal...,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
96625,"\nsns.set(rc={'figure.figsize':(12,8)})\npalet...",1,0,1,0,0,0,0
96626,"sns.set(rc={'figure.figsize':(12,8)})\npalette...",1,0,1,0,0,0,0
96627,"\nclf_list = [('Perceptron', Perceptron(eta0=0...",0,0,1,0,0,0,1
96628,"\nmodel_names, test_acc, test_f1 = test_set_pe...",0,0,0,0,0,0,1


Обозначим вершины графа знаний:

In [24]:
targets = ['imports', 'loaddata', 'visualize', 'preprocessing', 'model', 'train', 'predict']

Разделение на обучающую, тестовую и валидационную выборки, а также переработка сниппетов в файлы под требования проекта code2vec:

In [25]:
from sklearn.model_selection import train_test_split
SEED = 42001

X_train, X_test, y_train, y_test = train_test_split(ready_data.drop(targets, axis=1),
                                                   ready_data[targets],
                                                   test_size=3/11, random_state=SEED)

X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
                                               test_size=0.5, random_state=SEED)

In [26]:
train_set = X_train.reset_index().drop(columns=['index'], axis=1)
test_set = X_test.reset_index().drop(columns=['index'], axis=1)
valid_set = X_val.reset_index().drop(columns=['index'], axis=1)

In [27]:
y_train = y_train.reset_index().drop(columns=['index'], axis=1)
y_test = y_test.reset_index().drop(columns=['index'], axis=1)
y_val = y_val.reset_index().drop(columns=['index'], axis=1)

Далее все '\n' заменяются на '\n\t', а также вначале каждого сниппета приписывается

'def tag():\n\t'

Каждый сниппет записывается в **отдельный файл** с названием номер_итерации.py в каталог трей, тест или валидейшн.

In [21]:
def add_tabs(exmpl):
    new_exmpl = str()
    for i in range(len(exmpl)):
        if (exmpl[i]=='\n'):
            new_exmpl += '\n\t'
        else:
            new_exmpl += exmpl[i]
    return new_exmpl

In [61]:
def func_name(target):
    name = ''
    if target[3] == 1:
        return 'preprocessing_block'
    else:
        return 'no_name_block'

train:

In [63]:
for i in range(train_set.shape[0]):
        exmpl = train_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + func_name(y_train.loc[i]) +'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_train/func'+str(i)+'.py', 'w'))

test:

In [64]:
for i in range(test_set.shape[0]):
        exmpl = test_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + func_name(y_test.loc[i]) +'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_test/func'+str(i)+'.py', 'w'))

valid:

In [65]:
for i in range(valid_set.shape[0]):
        exmpl = valid_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + func_name(y_val.loc[i]) +'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_val/func'+str(i)+'.py', 'w'))

Дальше загружаем и распределяем для code2vec валидационную выборку, размеченную экспертно:

In [16]:
validation = pd.read_csv('/home/kirill/Desktop/all_google_sheet.csv')
validation = validation['Code']
validation = validation[validation.isna() != True]

In [18]:
validation = validation.reset_index().drop(columns=['index'], axis=1)

In [19]:
to_del = []
for i in range(validation.shape[0]):
    if 'def' in validation.loc[i]['Code']:
        to_del.append(i)

validation = validation.drop(index=to_del, axis=1)
validation = validation.reset_index().drop(columns=['index'], axis=1)
validation

,Code
0,for dataset in full_data:\n dataset['Embark...
1,for dataset in data_cleaner:\n dataset[...
2,label = LabelEncoder()\nfor dataset in data_cl...
3,for dataset in combine:\n dataset['Title'] ...
4,"title_mapping = {""Mr"": 1, ""Miss"": 2, ""Mrs"": 3,..."
...,...
181,from sklearn.feature_extraction.text import Co...
182,\nfrom textblob import TextBlob\n\ndf['sentime...
183,\n\n\n\nimport numpy as np\nfrom gensim.models...
184,from gensim.scripts.glove2word2vec import glov...


In [22]:
for i in range(validation.shape[0]):
    exmpl = validation.loc[i]['Code']
    exmpl = add_tabs(exmpl)
    print('def ' + 'f' +'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/real_validation/func'+str(i)+'.py', 'w'))